In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [2]:
%run project.py

In [3]:
num_meters = 96
input_dim = num_meters
hidden_dim = 96
batch_dim = 1
output_dim = 96 * 4
num_layers = 2
learning_rate = 0.001
num_epochs = 1

In [4]:
ami = pd.read_csv('./data/ami-uniq.txt', sep='\t')
ami['datetime'] = pd.to_datetime(ami['datetime'], format='%Y-%m-%d %H:%M:%S')
#weather = pd.read_csv('./data/WeatherData_Zipcode_77088.csv')
#weather = weather.drop(columns=['LocalDateTime', 'TimeZone'])

In [5]:
#ami.loc[ami['id']==81].plot(y='kw', figsize=(30,6))

In [6]:
series_kw = ami.pivot(index='id', columns='datetime', values='kw')

In [7]:
#series_kw.count().unique()

In [8]:
t_0 = series_kw.loc[:,series_kw.count() > 82].columns[0]
t_f = series_kw.loc[:,series_kw.count() > 82].columns[-1]
kw_part = series_kw.loc[:,t_0:t_f].fillna(series_kw.mean())
kw_x_norm = kw_part / (kw_part.max() - kw_part.min())
kw_sum = kw_part.sum() / kw_part.count() * num_meters
kw_y_norm = kw_sum / (kw_sum.max() - kw_sum.min())

In [9]:
#kw_y_norm.plot(figsize=(100,5))

In [10]:
X_train = torch.tensor(kw_x_norm.transpose().values).float()
X_train = X_train.view([-1, 1, input_dim])
y_train = torch.tensor(kw_y_norm).float()
y_train = y_train.unfold(0, 96*4, 1)
#y_train.size()

In [11]:
model = AmiTest(input_dim, hidden_dim, batch_dim, output_dim, num_layers)
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
hist = np.zeros(num_epochs)

In [12]:
model.hidden = model.init_hidden()

In [12]:
for t in range(num_epochs):
    y_pred = model(X_train)
    loss = loss_fn(y_pred[0:y_train.size(0),0], y_train)
    if t % 100 == 0:
        print("Epoch ", t, "MSE: ", loss.item())
    hist[t] = loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Epoch  0 MSE:  nan


In [13]:
y_pred = model(X_train)
loss = loss_fn(y_pred[0:y_train.size(0),0], y_train)

In [14]:
y_pred

tensor([[[ 0.0392,  0.0497, -0.0708,  ..., -0.0191, -0.0197, -0.0051]],

        [[ 0.0447,  0.0436, -0.0653,  ..., -0.0115, -0.0092, -0.0078]],

        [[ 0.0493,  0.0392, -0.0638,  ..., -0.0084, -0.0007, -0.0088]],

        ...,

        [[    nan,     nan,     nan,  ...,     nan,     nan,     nan]],

        [[    nan,     nan,     nan,  ...,     nan,     nan,     nan]],

        [[    nan,     nan,     nan,  ...,     nan,     nan,     nan]]],
       grad_fn=<ThAddBackward>)

In [18]:
model.lstm_out

AttributeError: 'AmiTest' object has no attribute 'lstm_out'

In [ ]:
X_train.shape

In [17]:
plt.plot(y_pred[0:y_train.size(0),0][0].detach().numpy())

NameError: name 'y_pred' is not defined